This notebook was run in Google Colab. To run every cell you need to upload the data set and prompts into the same folder. A valid OpenAI API token is needed, and must be saved as a secret under the name OPENAI_API_KEY.

Conversations can be found in the folder "model_conversations"

In [1]:
from google.colab import userdata
# if opnai not installed uncomment pip
!pip install openai
from openai import OpenAI
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.5 MB/s eta 0:00:00


NotebookAccessError: Notebook does not have access to secret OPENAI_API_KEY

In [ ]:
# Define the system message
system_msg = """
You are a reviewer for chatbot systems. The goal of the chatbot is to provide emotional support to users who are experiencing emotional distress.

An ideal emotional support system is one that goes through the following stages in order to assist the user:
Exploration stage: It explores the user’s feelings by asking open questions that prompts the user to elaborate further. Exploration can be weak (“What happened”, or “Tell me more”) or it can be strong (“Does this make you feel anxious?”)
Insight stage: It provides insight by offering reassurance and affirmation, or by reflecting the feelings back. Insights can be weak (“Everything will be ok”) or strong (“I would feel very isolated if this would happen to me”)
Action stage: It helps the user to solve the problem at hand by offering solutions, or by giving helpful information. Actions can be weak (“You should stop being so stressed”) or strong (“Here are some ways to manage stress…”)

All responses should center around the user seeking help. The emotional support system should therefore NOT change the subject or try to distract the user unless the user specifically requests it.
An ideal emotional support system does NOT deny the other’s feelings and perspective by criticising or challenging their legitimacy, or by telling the other how he or she should act and feel.
It should also provide support through a conversational setting, meaning that only sending lists of what the user should do is HIGHLY discouraged as it is very impersonal.

You will read through a snippet, where a user (marked USER) is seeking support, and the chatbot (marked SUPPORTER) is responding.
Based on this snippet, you will answer questions on a scale of 1-10, where 1 is a strong disagree, while 10 is a strong agree. Only output the number which equals the score you give to the snippet.

"""

In [ ]:
def read_conversation(number):
  convo = ""
  with open(f"conversations/convo{number}.txt", 'r') as file:
    for row in file:
      convo += row

  return convo



In [ ]:
questions = {
    "Q1": "Are the responses relevant and specific to the situation?",
    "Q2": "Does the conversation style (especially when in comes to the responses of the supporter) flow like a normal chat between two people (e.g. no excessive use of lists, responses are not too long)?",
    "Q3": "Are the ideas and intentions being relayed by the supporter ones you would expect to come from a human (regardless of phrasing)?",
    "Q4": "Are the responses brief and concise while also being helpful? (preferably not longer than 2 sentences)",
    "Q5": "Are the responses from the supporter calm and comforting? I.e. no extreme reactions.",
    "Q6": "Do the responses show respect for the user’s background (culture, identity…)?",
    "Q7": "Do the responses show sensitivity and empathy to the issue in question?",
    "Q8": "Does the chatbot encourage the user to express their thoughts and feelings openly?",
    "Q9": "Does the chatbot prompt the user to clarify and elaborate on their thoughts and feelings, leading to deeper self-exploration?",
    "Q10": "Does the chatbot helps the user to connect their thoughts and emotions to understand their experiences better?",
    "Q11": "Does the chatbot assist in recognizing patterns in the user’s thoughts, feelings, and/or behaviors?",
    "Q12": "Does the chatbot introduce new perspectives that challenge the user’s existing viewpoints, fostering insight?",
    "Q13": "Does the chatbot aid in developing coping strategies for managing the user’s emotional challenges?",
    "Q14": "Does the chatbot support the user in setting actionable and realistic goals for making changes in their thoughts, feelings, or behaviors?"
}

In [ ]:
convo = read_conversation(1)
print(convo)

USER: I lost someone very close to me recently, and I just feel so lost myself now.
SUPPORTER:  I'm so sorry for your loss. Grief can be an overwhelming experience, but know that with time and support, the pain will become more manageable. Please don't hesitate to reach out to a mental health professional who can guide you through this difficult process.
USER: It’s like I’m in this constant state of sadness. Sometimes, I even find myself forgetting they're gone, and then it hits me all over again.
SUPPORTER:  Waves of intense grief and disbelief are common after a significant loss. Be gentle with yourself during this time, and consider joining a support group to connect with others who understand what you're going through. Seeking professional counseling can also provide tools to cope with the ups and downs of the grieving process.
USER: I haven't considered a support group, it might be good to talk to others who understand.
SUPPORTER:  I'm glad you're open to exploring a support group

In [ ]:
client = OpenAI()

total_responses = list()


for i in range(10):
  convo = read_conversation(i+1)
  gpt_responses = list()

  # Define the user message
  for question_no in questions:

      user_msg = convo + "\n" + questions[question_no]


      response = client.chat.completions.create(
        model="gpt-4-0125-preview",
        #response_format={ "type": "json_object" },
        messages=[
          {"role": "system", "content": system_msg},
          {"role": "user", "content": user_msg}
        ]
      )
      gpt_responses.append(response.choices[0].message.content)

  total_responses.append(gpt_responses)



In [ ]:
import csv

with open("gpt_responses.txt", "w") as file1:
    writer = csv.writer(file1)
    # Writing data to a file
    for response in total_responses:

        int_scores = [int(float(score)) for score in response]
        print(int_scores)
        writer.writerow(int_scores)

# for i, convo in enumerate(total_responses):
#   #print("Convo number: ", i+1)

#   print(convo)



[10, 10, 10, 3, 10, 10, 10, 10, 9, 9, 9, 8, 10, 8]
[10, 10, 10, 5, 10, 10, 10, 10, 9, 9, 8, 8, 10, 8]
[10, 8, 10, 2, 10, 10, 10, 10, 8, 9, 7, 8, 9, 10]
[10, 10, 10, 5, 10, 10, 10, 10, 7, 8, 7, 5, 9, 8]
[10, 9, 10, 2, 10, 10, 10, 10, 8, 9, 10, 10, 10, 10]
[10, 9, 10, 2, 10, 10, 10, 9, 7, 8, 8, 8, 10, 10]
[10, 10, 10, 5, 10, 10, 10, 10, 8, 9, 8, 5, 10, 10]
[10, 10, 10, 4, 10, 10, 10, 10, 9, 9, 9, 8, 10, 10]
[10, 10, 10, 7, 10, 10, 10, 10, 9, 9, 8, 9, 9, 9]
[10, 9, 10, 5, 10, 10, 10, 10, 9, 9, 10, 7, 10, 10]


In [ ]:
# Testing scores
import numpy as np

# ind 0: general score
# ind 1: exploration score
# ind 2: insight score
# ind 3: action score
# ind 4: total score
scores = np.zeros(5)

total_lines = len(total_responses)
print(total_lines)

for response in total_responses:

    int_scores = [int(score) for score in response]

    general = np.average(int_scores[:7])
    expl = np.average(int_scores[7:9])
    insight = np.average(int_scores[9:12])
    action = np.average(int_scores[12:])

    score = 4 / (1/general + 1/expl + 1/insight + 1/action)

    scores[0] += general
    scores[1] += expl
    scores[2] += insight
    scores[3] += action
    scores[4] += score
    print(f"General: {general:.2f}, expl: {expl:.2f}, insight: {insight:.2f}, action: {action:.2f}, total: {score:.2f}")



scores /= total_lines

# Final output line shows the final scores used
print(f"General: {scores[0]:.2f}, expl: {scores[1]:.2f}, insight: {scores[2]:.2f}, action: {scores[3]:.2f}, total: {scores[4]:.2f}")




10
General: 9.00, expl: 9.50, insight: 8.67, action: 9.00, total: 9.03
General: 9.29, expl: 9.50, insight: 8.33, action: 9.00, total: 9.01
General: 8.57, expl: 9.00, insight: 8.00, action: 9.50, total: 8.73
General: 9.29, expl: 8.50, insight: 6.67, action: 8.50, total: 8.11
General: 8.71, expl: 9.00, insight: 9.67, action: 10.00, total: 9.32
General: 8.71, expl: 8.00, insight: 8.00, action: 10.00, total: 8.61
General: 9.29, expl: 9.00, insight: 7.33, action: 10.00, total: 8.79
General: 9.14, expl: 9.50, insight: 8.67, action: 10.00, total: 9.30
General: 9.57, expl: 9.50, insight: 8.67, action: 9.00, total: 9.17
General: 9.14, expl: 9.50, insight: 8.67, action: 10.00, total: 9.30
General: 9.07, expl: 9.10, insight: 8.27, action: 9.50, total: 8.94
